In [2]:
import sys
sys.path.append("/data/renhaoye/MorCG_DECaLS/")
sys.path.append("/data/renhaoye/MorCG_DECaLS/astre_utils/")
sys.path.append("/data/renhaoye/MorCG_DECaLS/astre_utils/catalog/")
sys.path.append("/data/renhaoye/MorCG_DECaLS/astre_utils/img/")
sys.path.append("/data/renhaoye/MorCG_DECaLS/data/")
sys.path.append("/data/renhaoye/MorCG_DECaLS/for_me/")
sys.path.append("/data/renhaoye/MorCG_DECaLS/models/")
sys.path.append("/data/renhaoye/MorCG_DECaLS/utils/")
sys.path.append("/data/renhaoye/MorCG_DECaLS/training/")

In [3]:
from astre_utils.catalog.match import match
from data.utils import load_dir
import numpy as np
from astropy.io import fits
import pandas as pd
from sklearn.model_selection import train_test_split
from utils.label_metadata import *
from utils.schemas import *
import os

In [4]:
"""local files in DECaLS"""
df_in_decals = load_dir("/data/renhaoye/MorCG/dataset/in_decals/raw_fits/")
"""local files in BASS+MzLS"""
df_out_decals = load_dir("/data/renhaoye/MorCG/dataset/out_decals/scaled/")
"""main galaxy sample"""
with fits.open("/data/renhaoye/MorCG/dataset/VAGC_MGS-m14_1777--20180116.fits") as hdul:
    ra = np.array(hdul[1].data["RA      "])
    dec = np.array(hdul[1].data["DEC     "])
    petro_mag = np.array(hdul[1].data["PETROMAG"][:, 2])
    MGS = pd.DataFrame(np.array((ra, dec, petro_mag)).T, columns=["ra", "dec", "petro_mag"])
# """overlap of DECaLS and BASS+MzLS"""
# overlap = match(df_in_decals, df_out_decals, 2, ["in_ra", "in_dec"]).drop(columns=["ra", "dec"]).rename(columns={"in_ra":"ra", "in_dec":"dec"})
df_c = pd.read_csv("/data/renhaoye/MorCG/dataset/gz_decals_volunteers_c.csv", index_col=0).rename(columns=lambda x: x.replace("-", "_")).fillna(0)

In [5]:
def get_each_question(df_c, votes):
    return df_c.query("smooth_or_featured_total_votes >= %d" % votes),\
           df_c.query("disk_edge_on_total_votes >= %d" % votes),\
           df_c.query("has_spiral_arms_total_votes >= %d" % votes),\
           df_c.query("bar_total_votes >= %d" % votes),\
           df_c.query("bulge_size_total_votes >= %d" % votes),\
           df_c.query("how_rounded_total_votes >= %d" % votes),\
           df_c.query("edge_on_bulge_total_votes >= %d" % votes),\
           df_c.query("spiral_winding_total_votes >= %d" % votes),\
           df_c.query("spiral_arm_count_total_votes >= %d" % votes),\
           df_c.query("merging_total_votes >= %d" % votes)
def get_total_question(df, votes, operator):
    if operator == ">=":
        return df.query("smooth_or_featured_total_votes >= %d | disk_edge_on_total_votes >= %d | has_spiral_arms_total_votes >= %d | bar_total_votes >= %d | bulge_size_total_votes >= %d | how_rounded_total_votes >= %d | edge_on_bulge_total_votes >= %d | spiral_winding_total_votes >= %d | spiral_arm_count_total_votes >= %d | merging_total_votes >= %d" %(votes, votes, votes, votes, votes, votes, votes, votes, votes, votes))
    if operator == "<=":
        return df.query("smooth_or_featured_total_votes >= 3 | disk_edge_on_total_votes >= 3 | has_spiral_arms_total_votes >= 3 | bar_total_votes >= 3 | bulge_size_total_votes >= 3 | how_rounded_total_votes >= 3 | edge_on_bulge_total_votes >= 3 | spiral_winding_total_votes >= 3 | spiral_arm_count_total_votes >= 3 | merging_total_votes >= 3").query("smooth_or_featured_total_votes <= %d | disk_edge_on_total_votes <= %d | has_spiral_arms_total_votes <= %d | bar_total_votes <= %d | bulge_size_total_votes <= %d | how_rounded_total_votes <= %d | edge_on_bulge_total_votes <= %d | spiral_winding_total_votes <= %d | spiral_arm_count_total_votes <= %d | merging_total_votes <= %d" %(votes, votes, votes, votes, votes, votes, votes, votes, votes, votes))

In [23]:
in_decals = match(df_in_decals, df_c, 2, ["real_ra", "real_dec"]).drop(columns=["ra", "dec"]).rename(columns={"real_ra":"ra", "real_dec":"dec"})
out_decals = match(df_out_decals, df_c, 2, ["real_ra", "real_dec"]).drop(columns=["ra", "dec"]).rename(columns={"real_ra":"ra", "real_dec":"dec"})
overlap = match(in_decals[["ra", "dec"]], out_decals, 2, ["in_ra", "in_dec"]).drop(columns=["ra", "dec"]).rename(columns={"in_ra":"ra", "in_dec":"dec"})
only_in_decals = pd.concat([in_decals, out_decals, out_decals]).drop_duplicates(keep=False)
only_out_decals = pd.concat([out_decals, in_decals, in_decals]).drop_duplicates(keep=False)
len(in_decals), len(out_decals), len(overlap), len(only_in_decals), len(only_out_decals)

(140931, 3713, 3618, 137313, 95)

In [12]:
only_in_decals_data = get_total_question(only_in_decals, 3, ">=")
len(only_in_decals_data), len(overlap)/len(only_in_decals_data)

(135599, 0.026681612696258822)

In [18]:
train, valid = train_test_split(only_in_decals_data, train_size=0.8266, random_state=1926)
valid = pd.concat([valid, overlap])
len(train), len(valid)

In [35]:
train.to_csv("/data/renhaoye/MorCG_DECaLS/dataset/train.csv")
valid.to_csv("/data/renhaoye/MorCG_DECaLS/dataset/valid.csv")

In [25]:
q1, q2, q3, q4, q5, q6, q7, q8, q9, q10 = get_each_question(valid, 34)
len(q1), len(q2), len(q3), len(q4), len(q5), len(q6), len(q7), len(q8), len(q9), len(q10)

(8058, 1183, 873, 873, 873, 1501, 119, 599, 599, 6763)

In [27]:
question_answer_pairs = gz2_pairs
dependencies = gz2_and_decals_dependencies
schema = Schema(question_answer_pairs, dependencies)
label_cols = schema.label_cols
def get_galaxy_label(galaxy, label_cols):
    # no longer casts to int64, user now responsible in df. If dtype is mixed, will try to infer with infer_objects
    return galaxy[label_cols].infer_objects().values.squeeze()  # squeeze for if there's one label_col

In [34]:
func = ["train", "valid"]
dataset = [train, valid]
# for i in range(1):
for i in range(len(func)):
    with open("/data/renhaoye/MorCG_DECaLS/dataset/mw_%s.txt" % func[i] , "w") as w:
        # for j in range(2,3):
        for j in range(len(dataset[i])):
            ra, dec = str(dataset[i].iloc[j, 0]), str(dataset[i].iloc[j, 1]),   # 先拿坐标和标签，这一步仅适用于decals的，sdss匹配的不可以
            path = "/data/renhaoye/MorCG/dataset/in_decals/agmtn/" + ra + "_" + dec + ".fits"
            if os.path.exists(path):
                w.write(path + " label:")
                for v in get_galaxy_label(dataset[i].iloc[j], label_cols):
                    w.write(str(v) + " ")
                w.write(" \n")
                if i == 0:
                    if os.path.exists(path.split(".fits")[0] + "_rotated.fits"):
                        w.write(path.split(".fits")[0] + "_rotated.fits" + " label:")
                        for v in get_galaxy_label(dataset[i].iloc[j], label_cols):
                            w.write(str(v) + " ")
                        w.write(" \n")
                    if os.path.exists(path.split(".fits")[0] + "_flipped.fits"):
                        w.write(path.split(".fits")[0] + "_flipped.fits" + " label:")
                        for v in get_galaxy_label(dataset[i].iloc[j], label_cols):
                            w.write(str(v) + " ")
                        w.write(" \n")
                    if os.path.exists(path.split(".fits")[0] + "_shifted.fits"):
                        w.write(path.split(".fits")[0] + "_shifted.fits" + " label:")
                        for v in get_galaxy_label(dataset[i].iloc[j], label_cols):
                            w.write(str(v) + " ")
                        w.write(" \n")
            else:
                print(path.split(".fits")[0])